In [1]:
LOCAL = True # CHANGE THIS 
cosine = False # CHANGE THIS 
multi = False # monocluster or multicluster
mode = None # CHANGE THIS - bit_corruption or None

from datetime import date
today = date.today().strftime("%d_%m_%Y")
if LOCAL: 
    checkpoint_folder = 'checkpoints/{}/'.format(today)
    try:
        os.makedirs(checkpoint_folder)
    except:
        print('checkpoint_folder already exists')
        
    if mode == 'bit_corruption':
        base_path = 'USPTO_50k_data/clean_rxn_50k_sparse_rxnFPs'
    else:
        base_path = 'USPTO_50k_data/clean_rxn_50k_sparse_FPs_numrcts'
    if cosine:
        if multi:
            cluster_path = 'USPTO_50k_data/50k_allmols_sparse_FP_clusterIndex.bin'
        else:
            cluster_path = 'USPTO_50k_data/50k_allmols_sparse_FP_MONOclusterIndex.bin'
        sparseFP_vocab_path = 'USPTO_50k_data/50k_all_mols_sparse_FPs.npz'
else: # colab 
    checkpoint_folder = '/content/gdrive/My Drive/rxn_ebm/checkpoints/{}/'.format(today) 
    try:
        os.makedirs(checkpoint_folder)
    except:
        print('checkpoint_folder already exists')
    if mode == 'bit_corruption':
        base_path = 'content/clean_rxn_50k_sparse_rxnFPs'
    else:
        base_path = 'content/clean_rxn_50k_sparse_FPs_numrcts'
    if cosine:
        if multi:
            cluster_path = 'content/50k_allmols_sparse_FP_clusterIndex.bin'
        else:
            cluster_path = 'content/50k_allmols_sparse_FP_MONOclusterIndex.bin'
        sparseFP_vocab_path = '/content/50k_all_mols_sparse_FPs.npz'

checkpoint_folder already exists


In [2]:
import pysparnn.cluster_index as ci

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/') 
# for Colab 
import os

import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG=True
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import rdChemReactions
from rdkit.Chem import rdqueries # faster than iterating atoms https://sourceforge.net/p/rdkit/mailman/message/34538007/ 
from rdkit.Chem.rdchem import Atom
from rdkit import DataStructs
import numpy as np

from itertools import chain
import random

from tqdm import tqdm
getattr(tqdm, '_instances', {}).clear() 
import pickle
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F

In [14]:
from experiment import Experiment
from model.FF import FeedforwardEBM

trainargs = {
    'model': 'FF_diff', # must change both model & fp_type 
    'hidden_sizes': [64],  
    'output_size': 1,
    'dropout': 0.1,  
    
    'batch_size': 64,
    'activation': 'ReLU',  
    'optimizer': torch.optim.Adam,
    'learning_rate': 8.5e-3, # to try: lr_finder & lr_schedulers 
    'epochs': 10,
    'early_stop': True,
    'min_delta': 1e-4, 
    'patience': 1,

    'checkpoint': True,
    'random_seed': 0, # only 1 seed is needed to seed all (torch, np, random etc.)
    
    'rctfp_size': 4096, # if fp_type == 'diff', ensure that both rctfp_size & prodfp_size are identical!
    'prodfp_size': 4096,
    'fp_radius': 3,
    'fp_type': 'diff',
    
    'num_neg_prod': 1,
    'num_neg_rct': 1,
    
    'base_path': base_path, # refer to top of notebook 
    'checkpoint_path': checkpoint_folder,
    'expt_name': 'testing',
    'device': torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

mode = None

In [15]:
model = FeedforwardEBM(trainargs)
experiment = Experiment(model, trainargs, mode=mode)

Using seed: 0


In [ ]:
experiment.train()